---
## Test self play

In [1]:
import numpy as np
import tensorflow as tf

In [2]:
from numpy import max as max_array_values
from numpy import sum as sum_array_values

In [3]:
from alphazero.game.config import config_2048
from alphazero.models.network import NetworkCacher
from alphazero.module.replay import ReplayBuffer
from alphazero.module.actor import StochasticMuZeroActor
from alphazero.train.self_play import run_self_play
from alphazero.addons.types import State

In [4]:
config = config_2048()
config.self_play.episodes = 1

In [5]:
replay_buffer = ReplayBuffer(config.replay)
cacher = NetworkCacher()
cacher.save_network(config.factory.network_factory())

2022-11-19 16:56:25.339990: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-11-19 16:56:25.340124: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1 Pro


In [6]:
run_self_play(config, cacher, replay_buffer)

Actor n°1: - Score: 890
Max: 512 for the self-play ...
Self-play took 10.37 minutes


In [12]:
len(replay_buffer._data)

1

In [8]:
replay_buffer._data[0][-1]

array([1.45807818e-04, 1.80712216e-04, 4.45606820e-04, 6.04059645e-04,
       8.84074306e-05, 2.21288321e-04, 7.15580925e-04, 4.20852227e-04,
       8.69942138e-05, 2.64056030e-03, 5.70374255e-04, 8.45125054e-05,
       9.73642880e-04, 4.27653761e-04, 6.47091800e-04, 5.40147864e-05,
       5.13580545e-04, 9.02222994e-04, 7.36111472e-04, 4.17429808e-04,
       3.82916710e-05, 6.24300351e-04, 2.60771819e-03, 5.29269802e-04,
       5.78487180e-03, 1.11129274e-04, 1.04951066e-03, 1.40280667e-04,
       1.75984583e-04, 2.43130838e-04, 3.58302432e-04, 4.28268757e-04,
       7.39981582e-04, 1.02439628e-04, 1.47809322e-03, 1.07122896e-03,
       7.13061270e-05, 7.15899670e-04, 5.29278182e-05, 3.21989328e-04,
       1.09099613e-03, 1.66100623e-03, 8.37568970e-04, 4.21685244e-03,
       2.09436877e-03, 6.69562444e-04, 5.78393715e-04, 1.77535269e-03,
       9.18593212e-04, 7.15039739e-04, 2.77145680e-03, 3.03103843e-03,
       5.29200114e-03, 6.63723960e-03, 4.80123634e-04, 9.24551510e-03,
      

In [9]:
%%timeit
replay_buffer.sample()

27.4 ms ± 102 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
from numpy import ndarray
from numba import prange, njit

In [11]:
all_values = np.array([state.search_stats.search_value for state in episode])

NameError: name 'episode' is not defined

In [ ]:
def _discount_values(all_values: ndarray, td_steps: int, td_lambda: float) -> ndarray:
    """
    Compute the discount values.

    Parameters
    ----------
    all_values: ndarray
        Value of each episode in trajectory
    td_steps:
        The number n of the n-step returns
    td_lambda:
        The lambda in TD(lambda)

    Returns
    -------
    ndarray
        Discount values
    """
    _len = all_values.shape[0]
    discount_values = np.zeros(_len)

    for i in prange(_len):
        discount_values[i] = all_values[i] * td_lambda ** td_steps

    return discount_values

In [ ]:
@njit(parallel=True)
def _discount_values_parallel(all_values: ndarray, td_steps: int, td_lambda: float) -> ndarray:
    """
    Compute the discount values.

    Parameters
    ----------
    all_values: ndarray
        Value of each episode in trajectory
    td_steps:
        The number n of the n-step returns
    td_lambda:
        The lambda in TD(lambda)

    Returns
    -------
    ndarray
        Discount values
    """
    _len = all_values.shape[0]
    discount_values = np.zeros(_len)

    for i in prange(_len):
        discount_values[i] = all_values[i] * td_lambda ** td_steps

    return discount_values

In [ ]:
@njit
def _discount_values_all(all_values: ndarray, td_steps: int, td_lambda: float) -> ndarray:
    """
    Compute the discount values.

    Parameters
    ----------
    all_values: ndarray
        Value of each episode in trajectory
    td_steps:
        The number n of the n-step returns
    td_lambda:
        The lambda in TD(lambda)

    Returns
    -------
    ndarray
        Discount values
    """
    _len = all_values.shape[0]
    discount_values = np.zeros(_len)

    for i in prange(_len):
        discount_values[i] = all_values[i] * td_lambda ** td_steps

    return discount_values

In [ ]:
%%timeit
_discount_values(all_values, 10, 0.5)

In [ ]:
%%timeit
_discount_values_parallel(all_values, 10, 0.5)

In [ ]:
%%timeit
_discount_values_all(all_values, 10, 0.5)

In [ ]:
@njit(cache=True)
def _sum_reward(rewards: ndarray, td_lambda: float) -> float:
    """
    Sum reward for N-step return.

    Parameters
    ----------
    rewards: ndarray
        List of rewards
    td_lambda: float
        Discount for n-step return

    Returns
    -------
    float
        n-step return
    """
    value = 0.0

    _len = rewards.shape[0]
    for i in prange(_len):
        value += rewards[i] * td_lambda ** i

    return value

@njit(cache=True)
def _discount_rewards(all_rewards: ndarray, td_steps: int, td_lambda: float) -> ndarray:
    """
    Compute the discount rewards.

    Parameters
    ----------
    all_rewards: ndarray
        Reward of each episode in trajectory
    td_steps:
        The number n of the n-step returns
    td_lambda:
        The lambda in TD(lambda)

    Returns
    -------
    ndarray
        All return of trajectories
    """
    _len = all_rewards.shape[0]
    discount_rewards = np.zeros(_len)

    for i in prange(_len):
        discount_rewards[i] = _sum_reward(all_rewards[i:i+td_steps], td_lambda)

    return discount_rewards

In [ ]:
all_rewards = np.array([state.reward for state in episode])

In [ ]:
%%timeit
_discount_rewards(all_rewards, 10, 0.5)

In [ ]:
all_visits = np.array([[state.search_stats.search_policy[a] if a in state.search_stats.search_policy else 0 for a in range(4)] for state in episode])

In [ ]:
@njit(cache=True)
def _policies(all_visits: ndarray):
    _len = all_visits.shape[0]
    policies = np.zeros(all_visits.shape)

    for i in prange(_len):
        total = np.sum(all_visits[i])
        policies[i] = all_visits[i] / total

    return policies

In [ ]:
%%timeit
_policies(all_visits)